### Openai

#### Load the API key and relevant Python libaries.

In [5]:
import openai
import os
import json 
import requests
from pprint import pprint
import json
import tiktoken
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
GPT_MODEL = "gpt-3.5-turbo-0613"

In [6]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
_ = load_dotenv(find_dotenv())
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [7]:
def count_token(input, model_name=GPT_MODEL, debug=False):
        encoding = tiktoken.encoding_for_model(model_name)
        codex = " ".join(input.splitlines())  # Join lines into a single string
        num_tokens = len(encoding.encode(codex))
        return num_tokens


### Completion Api

In [9]:
from openai import OpenAI
client = OpenAI()

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Write a tagline for an ice cream shop."
)

In [10]:
response.choices[0].text

'\n\n"Scoops of happiness in every cone!" '

### Chat Completions

In [11]:
client = OpenAI()

completion = client.chat.completions.create(
  model=GPT_MODEL,
  messages=[
    {"role": "system", "content": "You are an expert in writing taglines for ice cream shops. you are creative and original."},
    {"role": "user", "content": "Write a tagline for an ice cream shop."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='"Indulge in the Creamy Bliss, Scoop by Scoop!"', role='assistant', function_call=None, tool_calls=None)


pprint(completion.choices[0].message.content)

### Few shot Prompting 

In [11]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [12]:
response.choices[0].message.content

'The 2020 World Series was played in Arlington, Texas at Globe Life Field, the home stadium of the Texas Rangers.'

### delimiter and context

In [16]:
# articles = """List of 2022 Winter Olympics medal winners

# ==Curling==

# {{main|Curling at the 2022 Winter Olympics}}
# {|{{MedalistTable|type=Event|columns=1|width=225|labelwidth=200}}
# |-valign="top"
# |Men<br/>{{DetailsLink|Curling at the 2022 Winter Olympics – Men's tournament}}
# |{{flagIOC|SWE|2022 Winter}}<br/>[[Niklas Edin]]<br/>[[Oskar Eriksson]]<br/>[[Rasmus Wranå]]<br/>[[Christoffer Sundgren]]<br/>[[Daniel Magnusson (curler)|Daniel Magnusson]]
# |{{flagIOC|GBR|2022 Winter}}<br/>[[Bruce Mouat]]<br/>[[Grant Hardie]]<br/>[[Bobby Lammie]]<br/>[[Hammy McMillan Jr.]]<br/>[[Ross Whyte]]
# |{{flagIOC|CAN|2022 Winter}}<br/>[[Brad Gushue]]<br/>[[Mark Nichols (curler)|Mark Nichols]]<br/>[[Brett Gallant]]<br/>[[Geoff Walker (curler)|Geoff Walker]]<br/>[[Marc Kennedy]]
# |-valign="top"
# |Women<br/>{{DetailsLink|Curling at the 2022 Winter Olympics – Women's tournament}}
# |{{flagIOC|GBR|2022 Winter}}<br/>[[Eve Muirhead]]<br/>[[Vicky Wright]]<br/>[[Jennifer Dodds]]<br/>[[Hailey Duff]]<br/>[[Mili Smith]]
# |{{flagIOC|JPN|2022 Winter}}<br/>[[Satsuki Fujisawa]]<br/>[[Chinami Yoshida]]<br/>[[Yumi Suzuki]]<br/>[[Yurika Yoshida]]<br/>[[Kotomi Ishizaki]]
# |{{flagIOC|SWE|2022 Winter}}<br/>[[Anna Hasselborg]]<br/>[[Sara McManus]]<br/>[[Agnes Knochenhauer]]<br/>[[Sofia Mabergs]]<br/>[[Johanna Heldin]]
# |-valign="top"
# |Mixed doubles<br/>{{DetailsLink|Curling at the 2022 Winter Olympics – Mixed doubles tournament}}
# |{{flagIOC|ITA|2022 Winter}}<br/>[[Stefania Constantini]]<br/>[[Amos Mosaner]]
# |{{flagIOC|NOR|2022 Winter}}<br/>[[Kristin Skaslien]]<br/>[[Magnus Nedregotten]]
# |{{flagIOC|SWE|2022 Winter}}<br/>[[Almida de Val]]<br/>[[Oskar Eriksson]]
# |}

# Curling at the 2022 Winter Olympics

# ==Results summary==

# ===Women's tournament===

# ====Playoffs====

# =====Gold medal game=====

# ''Sunday, 20 February, 9:05''
# {{#lst:Curling at the 2022 Winter Olympics – Women's tournament|GM}}
# {{Player percentages
# | team1 = {{flagIOC|JPN|2022 Winter}}
# | [[Yurika Yoshida]] | 97%
# | [[Yumi Suzuki]] | 82%
# | [[Chinami Yoshida]] | 64%
# | [[Satsuki Fujisawa]] | 69%
# | teampct1 = 78%
# | team2 = {{flagIOC|GBR|2022 Winter}}
# | [[Hailey Duff]] | 90%
# | [[Jennifer Dodds]] | 89%
# | [[Vicky Wright]] | 89%
# | [[Eve Muirhead]] | 88%
# | teampct2 = 89%
# }}"""

prompts = """
# General Analysis Queries
""""What are the most important trends to note in the viewership data over the past month, considering views, watch time, and average view duration across different tables?""""
""""Which videos have performed best in the last quarter, based on data from the dim_top10 table?""""
""""How has the channel's subscriber growth rate changed over time, considering available data from all relevant tables?""""
""""What are the key differences in viewership patterns based on device type, traffic source, viewer demographics, and content type, leveraging insights from all tables?""""
""""Are there any correlations between specific video topics or formats and higher engagement metrics, as indicated by the data in dim_top10 and other tables?""""

# Specific Data Requests
""""Please provide a breakdown of views, watch time, and average view duration by device type for the past week, using data from raw_total and raw_gender tables.""""
""""Show me the top 5 traffic sources that are driving the most views to the channel, based on available data.""""
""""Identify the content types that have the highest average view duration, utilizing insights from dim_top10 and other relevant tables.""""
""""Compare the viewership demographics for subscribers versus non-subscribers, drawing upon data from raw_gender and other tables.""""
""""Analyze the impact of subtitles and closed captions on overall watch time, using data from raw_total and potentially other tables.""""

# SQL Query Generation
""""Generate the SQL query to retrieve the following data from the specified tables (clearly specify the desired data points and conditions).""""
""""Translate this natural language question into an SQL query that can be run against the database: (ask your question in plain English, specifying any relevant tables).""""

# Data Clarification
""""Could you please clarify what you mean by X, and which tables or columns are relevant to your query?""""
""""Could you provide an example of what you're looking for, specifying the tables and columns you'd like to focus on?""""
""""I'm not sure I understand the question fully. Could you rephrase it, and indicate any specific tables or columns you're interested in?""""

# Uncertainty Handling
""""I'm not confident in providing an accurate answer to that question based on the available data in these tables. Would you like me to try generating an approximate response or seek further input?""""
""""I'm unable to generate a query for that question due to ambiguity or potential data limitations. Could you please rephrase it or provide more specific details, including relevant tables and columns?""""
"""


In [17]:
# system_message =f"""Use the provided articles delimited by xml tags to answer questions. If the answer cannot be found in the prompts, write I could not find an answer.
# prompt: < {prompts} > 
# """
system_message = f"""
<prompts>
{prompts}
</prompts>
"""

In [18]:
count_token(system_message)

487

In [23]:
user_message = """Query: What are the top 5 most viewed videos on the channel in the past month?"""

In [25]:
count_token(user_message)

19

In [26]:
from openai import OpenAI
def get_gpt_response(system_message, user_message):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": user_message
            }
        ],
        temperature=0,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content


In [27]:
response = get_gpt_response(system_message, user_message)

In [28]:
response

'SELECT video_title, views\nFROM dim_top10\nWHERE date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)\nORDER BY views DESC\nLIMIT 5;'